In [1]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interactive

# Define the reduced van der Waals equation function
def van_der_Waals_reduced(T, v):
    return (8*T)/(3*v - 1) - 3/(v**2)

# Function to plot the diagram with automatic rescaling of the axes
def plot_van_der_Waals(T=1.0):
    v = np.linspace(0.5, 4, 400)  # Avoiding division by zero or negative volumes
    P = van_der_Waals_reduced(T, v)

    plt.figure(figsize=(8, 5))
    plt.plot(v, P, label=f'Reduced Temperature (T) = {T}')
    plt.title('Reduced van der Waals Equation $P-V$ Diagram')
    plt.xlabel('Reduced Volume $(v)$')
    plt.ylabel('Reduced Pressure $(P)$')

    # Automatic rescaling of the y-axis
    P_min, P_max = P.min(), P.max()
    y_margin = (P_max - P_min) * 0.1  # Adding a 10% margin to the min and max of pressure values
    plt.ylim(P_min - y_margin, P_max + y_margin)

    plt.axhline(0, color='black', lw=1)  # Adding a horizontal line at P=0 for reference
    plt.axvline(1/3, color='red', linestyle='--', label='Excluded Volume')  # Excluded volume at v=1/3
    plt.legend()
    plt.grid(True)
    plt.show()

# Create an interactive widget for the temperature slider
interactive_plot = interactive(plot_van_der_Waals, T=(0.5, 2.0, 0.05))
output = interactive_plot.children[-1]
output.layout.height = '350px'
interactive_plot


interactive(children=(FloatSlider(value=1.0, description='T', max=2.0, min=0.5, step=0.05), Output(layout=Layo…

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize_scalar
from ipywidgets import interactive

# Define the reduced van der Waals equation function
def van_der_Waals_reduced(T, v):
    return (8*T)/(3*v - 1) - 3/(v**2)

# Area difference function for the Maxwell construction
def area_diff(P, T, v, P_vdw):
    # Splitting the volume range at the points where P equals the van der Waals pressure
    v_left, v_right = v[P_vdw > P], v[P_vdw <= P]
    area_left = np.trapz(P_vdw[P_vdw > P] - P, v_left)
    area_right = np.trapz(P - P_vdw[P_vdw <= P], v_right)
    return abs(area_left - area_right)

# Function to plot the diagram with Maxwell construction
def plot_van_der_Waals_with_Maxwell(T=0.85):
    v = np.linspace(0.5, 4, 400)
    P_vdw = van_der_Waals_reduced(T, v)

    if T < 1:  # Assuming Tc ~ 1 for reduced vdw equation, apply Maxwell construction for T < Tc
        # Find P_maxwell that minimizes the area difference
        res = minimize_scalar(area_diff, bounds=(min(P_vdw), max(P_vdw)), args=(T, v, P_vdw), method='bounded')
        P_maxwell = res.x

        # Plot the Maxwell construction line
        plt.axhline(P_maxwell, color='green', linestyle='--', label='Maxwell Construction')
    else:
        P_maxwell = None

    plt.figure(figsize=(8, 5))
    plt.plot(v, P_vdw, label=f'Reduced Temperature (T) = {T}')
    plt.title('Reduced van der Waals Equation $P-V$ Diagram')
    plt.xlabel('Reduced Volume $(v)$')
    plt.ylabel('Reduced Pressure $(P)$')

    # Automatic rescaling of the y-axis based on whether we have a Maxwell construction
    if P_maxwell is not None:
        plt.ylim(min(P_vdw) - 0.1, max(P_vdw) + 0.1)
    else:
        P_min, P_max = P_vdw.min(), P_vdw.max()
        y_margin = (P_max - P_min) * 0.1
        plt.ylim(P_min - y_margin, P_max + y_margin)

    plt.axhline(0, color='black', lw=1)
    plt.axvline(1/3, color='red', linestyle='--', label='Excluded Volume')
    plt.legend()
    plt.grid(True)
    plt.show()

# Create an interactive widget for the temperature slider
interactive_plot = interactive(plot_van_der_Waals_with_Maxwell, T=(0.5, 1.0, 0.01))
output = interactive_plot.children[-1]
output.layout.height = '350px'
interactive_plot


interactive(children=(FloatSlider(value=0.85, description='T', max=1.0, min=0.5, step=0.01), Output(layout=Lay…